# Building an Entity Normalization Engine

This notebook contains experiments in an attempt to build an entity normalization engine. The input to this engine is short strings that could encompass the following entities: company names, company addresses, serial numbers, physical goods and locations.

The task is divided into 2 parts:

**Part 1**: Build a system that can identify unique entities for each category above. Some of these will be trivial (remove spaces, edit distance) while others are more complicated and will need a trained model / some other form of knowledge and guidance.

**Part 2**: Build a system that receives strings one by one and groups together any entities that have passed through the system previously. Check the latest sample received, scan the entries already received, identify if the entity is a duplicate and then add it to a cluster / create a new cluster depending on the result.

Here are fictional examples of the strings we will be dealing with:

- Company names: “Marks and Spencers Ltd”, “M&S Limited”, “NVIDIA Ireland”
- Company addresses: “SLOUGH SE12 2XY”, “33 TIMBER YARD, LONDON, L1 8XY”, “44 CHINA ROAD, KOWLOON, HONG KONG”
- Serial numbers: “XYZ 13423 / ILD”, “ABC/ICL/20891NC”
- Physical Goods: “HARDWOOD TABLE”, “PLASTIC BOTTLE”
- Locations: “LONDON”, “HONG KONG”, “ASIA”

## Part 1: Category-Specific Normalization Engines

The general, theoretical approach here is:

*Step 1* - Text preprocessing: standardizing lettercase, punctuation, whitespace, accented/special characters, legal control terms (Ltd, Co, etc) depending on which category we are dealing with

*Step 2* - Entity clustering: use string_grouper library that uses TF-IDF (Term Frequency multiplied by Inverse Document Frequency) with N-Grams to calculate cosine similarities within a single Series of strings and groups them by assigning to each string one string from the group chosen as the group-representative for each group of similar strings found.

The code for both these steps will vary depending on the category we are dealing with. We can address the categories one by one in ascending order of difficulty. Functions are created for each in order to copy them to python scripts efficiently.

*Note: Original idea was to create for each category string similarity matrix with similarity metric such as levenshtein distance, Jaro-Winkler or caverphone algorithm. Next, use clustering algorithm to cluster entities; affinity propagation seems to be standard for this task. For each unique entity (cluster) assign substring with the longest string length as the standard name for that cluster. I experimented with these approach but found the chosen approach to be quicker to execute, more legible and have better performance.*

### Serial Numbers

We expect minimal pre-processing to be done for serial numbers and rely on a very high similarity threshold for matching strings. It's actually better to just group the strings directly after pre-processing, but we might as well just use the string grouper function to keep the process standard for all categories.

In [1]:
import pandas as pd

# create a sample df for serial numbers from manually generated list
serial_nums = ["XYZ 13423 / ILD", "ABC/ICL/20891NC", "XYZ-13423-ILD", "ABCICL-20891NC", "XYZ 14 / IKR", "DNC/ICL/20891NC", "XYK-13423-ILD"]
df_serial_nums = pd.DataFrame(serial_nums, columns=['serial_nums'])

In [2]:
df_serial_nums

,serial_nums
0,XYZ 13423 / ILD
1,ABC/ICL/20891NC
2,XYZ-13423-ILD
3,ABCICL-20891NC
4,XYZ 14 / IKR
5,DNC/ICL/20891NC
6,XYK-13423-ILD


In [5]:
import re
import numpy as np

# remove special characters function
def remove_special_characters(text):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

# preprocess function for dataframe of serial numbers - provide dataframe and column name for serial nums as input
def preprocess_serial_nums(dataframe, column_name):
    clean_list = [remove_special_characters(text) for text in dataframe[column_name]] 
    clean_list = [text.lower() for text in clean_list]
    clean_list = [text.replace(' ', '') for text in clean_list] # remove all whitespaces
    df_clean_serial_nums = pd.DataFrame(np.column_stack([dataframe[column_name], clean_list]), columns=['serial_nums', 'clean_serial_nums'])
    return df_clean_serial_nums

In [6]:
df_serial_nums_cleaned = preprocess_serial_nums(df_serial_nums, 'serial_nums'); df_serial_nums_cleaned

,serial_nums,clean_serial_nums
0,XYZ 13423 / ILD,xyz13423ild
1,ABC/ICL/20891NC,abcicl20891nc
2,XYZ-13423-ILD,xyz13423ild
3,ABCICL-20891NC,abcicl20891nc
4,XYZ 14 / IKR,xyz14ikr
5,DNC/ICL/20891NC,dncicl20891nc
6,XYK-13423-ILD,xyk13423ild


In [7]:
from string_grouper import group_similar_strings

# entity matching function for dataframe created at preprocessing stage - provide dataframe and original column name for serial nums
def entity_matcher_serial_nums(dataframe, column_name):
    # group entities using tf-idf to calculate cosine similarities - look for 99% minimum similarity
    dataframe[['unique_entity_ID', 'unique_entity']] = group_similar_strings(dataframe['clean_serial_nums'], min_similarity = 0.99)
    dataframe = dataframe.groupby(['unique_entity_ID', 'unique_entity'])[column_name].apply('; '.join).reset_index()
    return dataframe

In [8]:
df_unique_entities_serial_nums = entity_matcher_serial_nums(df_serial_nums_cleaned, 'serial_nums'); df_unique_entities_serial_nums

,unique_entity_ID,unique_entity,serial_nums
0,0,xyz13423ild,XYZ 13423 / ILD; XYZ-13423-ILD
1,1,abcicl20891nc,ABC/ICL/20891NC; ABCICL-20891NC
2,4,xyz14ikr,XYZ 14 / IKR
3,5,dncicl20891nc,DNC/ICL/20891NC
4,6,xyk13423ild,XYK-13423-ILD


In [9]:
import time

# function for processing df of serial numbers and clustering them using string similarity
def clustering_serial_nums(dataframe, column_name):
    # preprocess dataframe
    df_serial_nums_cleaned = preprocess_serial_nums(dataframe, column_name)
    # start clustering of serial nums
    print('Entity clustering will start now.')
    start = time.time()
    df_unique_entities_serial_nums = entity_matcher_serial_nums(df_serial_nums_cleaned, column_name)
    time.sleep(1)
    end = time.time()
    # print total time taken
    print(f"Runtime of the program is {end - start} seconds.")
    # create csv with timestamp
    current_time = time.strftime("%Y%m%d-%H%M%S")
    filename = 'serial_num_normalized' + str(current_time) + '.csv'
    # print job status
    print(f"{filename} has been saved in current directory.")
    return df_unique_entities_serial_nums.to_csv(filename, index = False)

In [10]:
clustering_serial_nums(df_serial_nums, 'serial_nums')

Entity clustering will start now.
Runtime of the program is 1.0202109813690186 seconds.
serial_num_normalized20210725-235347.csv has been saved in current directory.


In [11]:
# inspect results
pd.read_csv('serial_num_normalized20210725-235347.csv')

,unique_entity_ID,unique_entity,serial_nums
0,0,xyz13423ild,XYZ 13423 / ILD; XYZ-13423-ILD
1,1,abcicl20891nc,ABC/ICL/20891NC; ABCICL-20891NC
2,4,xyz14ikr,XYZ 14 / IKR
3,5,dncicl20891nc,DNC/ICL/20891NC
4,6,xyk13423ild,XYK-13423-ILD


### Company Addresses

This category is actually quite tricky, but we won't attempt to dive too deep into it for this exercise. We can simplify things and pretend that addresses can be addressed similarly to serial numbers. In reality address parsing and normalization is extremely tricky especially because even the simplest addresses are packed with local conventions, abbreviations and context. If we had to address this head-on we would use the [libpostal](https://github.com/openvenues/libpostal) library. It converts free-form addresses that humans use into clean normalized forms suitable for machine comparison and full-text indexing.

In [12]:
# create a sample df for addresses from manually generated list
addresses = ["SLOUGH SE12 2XY", "33 TIMBER YARD, LONDON, L1 8XY", "44 CHINA ROAD, KOWLOON, HONG KONG", "33, TIMBER YARD, LONDON, L1 8XY"]
df_addresses = pd.DataFrame(addresses, columns=['addresses'])

In [13]:
df_addresses

,addresses
0,SLOUGH SE12 2XY
1,"33 TIMBER YARD, LONDON, L1 8XY"
2,"44 CHINA ROAD, KOWLOON, HONG KONG"
3,"33, TIMBER YARD, LONDON, L1 8XY"


In [14]:
# preprocess function for dataframe of addresses - provide dataframe and column name for addresses as input
def preprocess_addresses(dataframe, column_name):
    clean_list = [remove_special_characters(text) for text in dataframe[column_name]] 
    clean_list = [text.lower() for text in clean_list]
    clean_list = [text.replace('  ', ' ') for text in clean_list] # remove extra whitespaces
    clean_list = [text.strip() for text in clean_list]
    df_clean_addresses = pd.DataFrame(np.column_stack([dataframe[column_name], clean_list]), columns=['addresses', 'clean_addresses'])
    return df_clean_addresses

In [15]:
df_addresses_cleaned = preprocess_addresses(df_addresses, 'addresses'); df_addresses_cleaned

,addresses,clean_addresses
0,SLOUGH SE12 2XY,slough se12 2xy
1,"33 TIMBER YARD, LONDON, L1 8XY",33 timber yard london l1 8xy
2,"44 CHINA ROAD, KOWLOON, HONG KONG",44 china road kowloon hong kong
3,"33, TIMBER YARD, LONDON, L1 8XY",33 timber yard london l1 8xy


In [16]:
# entity matching function for dataframe created at preprocessing stage - provide dataframe and original column name for addresses
def entity_matcher_addresses(dataframe, column_name):
    # group entities using tf-idf to calculate cosine similarities - look for 99% minimum similarity
    dataframe[['unique_entity_ID', 'unique_entity']] = group_similar_strings(dataframe['clean_addresses'], min_similarity = 0.99)
    dataframe = dataframe.groupby(['unique_entity_ID', 'unique_entity'])[column_name].apply('; '.join).reset_index()
    return dataframe

In [17]:
df_unique_entities_addresses = entity_matcher_addresses(df_addresses_cleaned, 'addresses'); df_unique_entities_addresses

,unique_entity_ID,unique_entity,addresses
0,0,slough se12 2xy,SLOUGH SE12 2XY
1,1,33 timber yard london l1 8xy,"33 TIMBER YARD, LONDON, L1 8XY; 33, TIMBER YAR..."
2,2,44 china road kowloon hong kong,"44 CHINA ROAD, KOWLOON, HONG KONG"


In [18]:
# function for processing df of addresses and clustering them using string similarity
def clustering_addresses(dataframe, column_name):
    # preprocess dataframe
    df_addresses_cleaned = preprocess_addresses(dataframe, column_name)
    # start clustering of addresses
    print('Entity clustering will start now.')
    start = time.time()
    df_unique_entities_addresses = entity_matcher_addresses(df_addresses_cleaned, column_name)
    time.sleep(1)
    end = time.time()
    # print total time taken
    print(f"Runtime of the program is {end - start} seconds.")
    # create csv with timestamp
    current_time = time.strftime("%Y%m%d-%H%M%S")
    filename = 'addresses_normalized' + str(current_time) + '.csv'
    # print job status
    print(f"{filename} has been saved in current directory.")
    return df_unique_entities_addresses.to_csv(filename, index = False)

In [19]:
clustering_addresses(df_addresses, 'addresses')

Entity clustering will start now.
Runtime of the program is 1.0151150226593018 seconds.
addresses_normalized20210725-235415.csv has been saved in current directory.


In [20]:
# inspect results
pd.read_csv('addresses_normalized20210725-235415.csv')

,unique_entity_ID,unique_entity,addresses
0,0,slough se12 2xy,SLOUGH SE12 2XY
1,1,33 timber yard london l1 8xy,"33 TIMBER YARD, LONDON, L1 8XY; 33, TIMBER YAR..."
2,2,44 china road kowloon hong kong,"44 CHINA ROAD, KOWLOON, HONG KONG"


### Physical Goods

For this category we can start using similarity thresholds that are lower. We start getting into entities that would make more sense to use a language model with. We'll stick with the TF-IDF cosine similarity approach for the sake of this exercise and see if it does the job.

In [21]:
# create a sample df for physical goods from manually generated list
goods = ["HARDWOOD TABLE", "PLASTIC BOTTLE", "TOYS", "plastic water bottle", "Wooden Table", "ALUMINIUM TABLE"]
df_goods = pd.DataFrame(goods, columns=['goods'])

In [22]:
df_goods

,goods
0,HARDWOOD TABLE
1,PLASTIC BOTTLE
2,TOYS
3,plastic water bottle
4,Wooden Table
5,ALUMINIUM TABLE


In [23]:
# preprocess function for dataframe of goods - provide dataframe and column name for goods as input
def preprocess_goods(dataframe, column_name):
    clean_list = [remove_special_characters(text) for text in dataframe[column_name]] 
    clean_list = [text.lower() for text in clean_list]
    clean_list = [text.replace('  ', ' ') for text in clean_list] # remove extra whitespaces
    clean_list = [text.strip() for text in clean_list]
    df_clean_goods = pd.DataFrame(np.column_stack([dataframe[column_name], clean_list]), columns=['goods', 'clean_goods'])
    return df_clean_goods

In [24]:
df_goods_cleaned = preprocess_goods(df_goods, 'goods'); df_goods_cleaned

,goods,clean_goods
0,HARDWOOD TABLE,hardwood table
1,PLASTIC BOTTLE,plastic bottle
2,TOYS,toys
3,plastic water bottle,plastic water bottle
4,Wooden Table,wooden table
5,ALUMINIUM TABLE,aluminium table


In [25]:
# entity matching function for dataframe created at preprocessing stage - provide dataframe and original column name for goods
def entity_matcher_goods(dataframe, column_name):
    # group entities using tf-idf to calculate cosine similarities - look for 30% minimum similarity
    dataframe[['unique_entity_ID', 'unique_entity']] = group_similar_strings(dataframe['clean_goods'], min_similarity = 0.3)
    dataframe = dataframe.groupby(['unique_entity_ID', 'unique_entity'])[column_name].apply('; '.join).reset_index()
    return dataframe

In [26]:
df_unique_entities_goods = entity_matcher_goods(df_goods_cleaned, 'goods'); df_unique_entities_goods

,unique_entity_ID,unique_entity,goods
0,0,hardwood table,HARDWOOD TABLE; Wooden Table
1,1,plastic bottle,PLASTIC BOTTLE; plastic water bottle
2,2,toys,TOYS
3,5,aluminium table,ALUMINIUM TABLE


In [27]:
# function for processing df of goods and clustering them using string similarity
def clustering_goods(dataframe, column_name):
    # preprocess dataframe
    df_goods_cleaned = preprocess_goods(dataframe, column_name)
    # start clustering of goods
    print('Entity clustering will start now.')
    start = time.time()
    df_unique_entities_goods = entity_matcher_goods(df_goods_cleaned, column_name)
    time.sleep(1)
    end = time.time()
    # print total time taken
    print(f"Runtime of the program is {end - start} seconds.")
    # create csv with timestamp
    current_time = time.strftime("%Y%m%d-%H%M%S")
    filename = 'goods_normalized' + str(current_time) + '.csv'
    # print job status
    print(f"{filename} has been saved in current directory.")
    return df_unique_entities_goods.to_csv(filename, index = False)

In [28]:
clustering_goods(df_goods, 'goods')

Entity clustering will start now.
Runtime of the program is 1.0178301334381104 seconds.
goods_normalized20210725-235445.csv has been saved in current directory.


In [29]:
# inspect results
pd.read_csv('goods_normalized20210725-235445.csv')

,unique_entity_ID,unique_entity,goods
0,0,hardwood table,HARDWOOD TABLE; Wooden Table
1,1,plastic bottle,PLASTIC BOTTLE; plastic water bottle
2,2,toys,TOYS
3,5,aluminium table,ALUMINIUM TABLE


### Locations

Once again, it would make more sense to use a language model for this task, but since the TF-IDF cosine similarity approach performed okay with physical goods, we can definitely use it for locations.

In [30]:
# create a sample df for locations from manually generated list
locations = ["LONDON", "HONG KONG", "ASIA", "LONDON, ENG", "LONDON, GREAT BRITAIN", "LONDON, ENGLAND"]
df_locations = pd.DataFrame(locations, columns=['locations'])

In [31]:
df_locations

,locations
0,LONDON
1,HONG KONG
2,ASIA
3,"LONDON, ENG"
4,"LONDON, GREAT BRITAIN"
5,"LONDON, ENGLAND"


In [32]:
# preprocess function for dataframe of locations - provide dataframe and column name for locations as input
def preprocess_locations(dataframe, column_name):
    clean_list = [remove_special_characters(text) for text in dataframe[column_name]] 
    clean_list = [text.lower() for text in clean_list]
    clean_list = [text.replace('  ', ' ') for text in clean_list] # remove extra whitespaces
    clean_list = [text.strip() for text in clean_list]
    df_clean_locations = pd.DataFrame(np.column_stack([dataframe[column_name], clean_list]), columns=['locations', 'clean_locations'])
    return df_clean_locations

In [33]:
df_locations_cleaned = preprocess_locations(df_locations, 'locations'); df_locations_cleaned

,locations,clean_locations
0,LONDON,london
1,HONG KONG,hong kong
2,ASIA,asia
3,"LONDON, ENG",london eng
4,"LONDON, GREAT BRITAIN",london great britain
5,"LONDON, ENGLAND",london england


In [34]:
# entity matching function for dataframe created at preprocessing stage - provide dataframe and original column name for locations
def entity_matcher_locations(dataframe, column_name):
    # group entities using tf-idf to calculate cosine similarities - look for 30% minimum similarity
    dataframe[['unique_entity_ID', 'unique_entity']] = group_similar_strings(dataframe['clean_locations'], min_similarity = 0.3)
    dataframe = dataframe.groupby(['unique_entity_ID', 'unique_entity'])[column_name].apply('; '.join).reset_index()
    return dataframe

In [35]:
df_unique_entities_locations = entity_matcher_locations(df_locations_cleaned, 'locations'); df_unique_entities_locations

,unique_entity_ID,unique_entity,locations
0,0,london,"LONDON; LONDON, ENG; LONDON, GREAT BRITAIN; LO..."
1,1,hong kong,HONG KONG
2,2,asia,ASIA


In [36]:
# function for processing df of locations and clustering them using string similarity
def clustering_locations(dataframe, column_name):
    # preprocess dataframe
    df_locations_cleaned = preprocess_locations(dataframe, column_name)
    # start clustering of locations
    print('Entity clustering will start now.')
    start = time.time()
    df_unique_entities_locations = entity_matcher_locations(df_locations_cleaned, column_name)
    time.sleep(1)
    end = time.time()
    # print total time taken
    print(f"Runtime of the program is {end - start} seconds.")
    # create csv with timestamp
    current_time = time.strftime("%Y%m%d-%H%M%S")
    filename = 'locations_normalized' + str(current_time) + '.csv'
    # print job status
    print(f"{filename} has been saved in current directory.")
    return df_unique_entities_locations.to_csv(filename, index = False)

In [37]:
clustering_locations(df_locations, 'locations')

Entity clustering will start now.
Runtime of the program is 1.0146229267120361 seconds.
locations_normalized20210725-235520.csv has been saved in current directory.


In [38]:
# inspect results
pd.read_csv('locations_normalized20210725-235520.csv')

,unique_entity_ID,unique_entity,locations
0,0,london,"LONDON; LONDON, ENG; LONDON, GREAT BRITAIN; LO..."
1,1,hong kong,HONG KONG
2,2,asia,ASIA


### Company Names

The approach we used for all categories could probably work for company names, but it could be interesting to try out something that I think could work quite well for company names: scraping the first link that comes up when performing a google search for each company name we have. We can compare this with the approach we used until now and compare performance.

In [39]:
names = [
"Marks and Spencers Ltd", 
"M&S Limited",
"NVIDIA Ireland",
"Intel LLC",
"AT & T",
"AT & T MOBILITY",
"AT&T",
"AT&T MOBILITY",
"DELL INC.",
"WAL-MART COMMUNITY",
"WALMART PAYMENTS",
"PITNEY BOWES",
"PITNEY BOWES GLOBAL FINANCIAL",
"PITNEY BOWES, INC",
"ORACLE", 
"ORACLE AMERICA INC", 
"ORACLE CORPORATION", 
"ORACLE USA INC"]

In [40]:
df_names = pd.DataFrame(names, columns=['names'])

In [41]:
from googlesearch import search
    
# preprocess function for dataframe of locations - provide dataframe and column name for locations as input
def google_names(dataframe, column_name):
    clean_list = [list(search(text, tld="com", num=1, stop=1, pause=0.1))[0] for text in dataframe[column_name]] 
    df_clean_names = pd.DataFrame(np.column_stack([dataframe[column_name], clean_list]), columns=['comp_names', 'google_comp_names'])
    return df_clean_names

In [42]:
df_names_cleaned = google_names(df_names, 'names'); df_names_cleaned

,comp_names,google_comp_names
0,Marks and Spencers Ltd,https://www.marksandspencer.com/
1,M&S Limited,https://www.marksandspencer.com/ie/l/women/lim...
2,NVIDIA Ireland,https://shop.nvidia.com/en-us/
3,Intel LLC,https://software.intel.com/content/www/us/en/d...
4,AT & T,https://www.att.com/
5,AT & T MOBILITY,https://www.att.com/wireless/
6,AT&T,https://www.att.com/
7,AT&T MOBILITY,https://www.att.com/wireless/
8,DELL INC.,https://www.dell.com/
9,WAL-MART COMMUNITY,https://walmart.org/what-we-do/strengthening-c...


This approach is not giving great results and most of all it's extremely slow. It might be better to revert back to the original idea.

In [43]:
df_names = pd.DataFrame(names, columns=['names'])

In [44]:
from cleanco import cleanco

# preprocess function for dataframe of names - provide dataframe and column name for names as input
def preprocess_names(dataframe, column_name):
    clean_list = [remove_special_characters(text) for text in dataframe[column_name]] 
    clean_list = [text.lower() for text in clean_list]
    clean_list = [text.replace('  ', ' ') for text in clean_list] # remove extra whitespaces
    clean_list = [text.strip() for text in clean_list]
    clean_list = [cleanco(text).clean_name() for text in clean_list] # remove legal control terms
    df_clean_names = pd.DataFrame(np.column_stack([dataframe[column_name], clean_list]), columns=['names', 'clean_names'])
    return df_clean_names

In [45]:
df_names_cleaned = preprocess_names(df_names, 'names'); df_names_cleaned

,names,clean_names
0,Marks and Spencers Ltd,marks and spencers
1,M&S Limited,ms
2,NVIDIA Ireland,nvidia ireland
3,Intel LLC,intel
4,AT & T,at t
5,AT & T MOBILITY,at t mobility
6,AT&T,att
7,AT&T MOBILITY,att mobility
8,DELL INC.,dell
9,WAL-MART COMMUNITY,walmart community


In [46]:
# entity matching function for dataframe created at preprocessing stage - provide dataframe and original column name for names
def entity_matcher_names(dataframe, column_name):
    # group entities using tf-idf to calculate cosine similarities - look for 20% minimum similarity
    dataframe[['unique_entity_ID', 'unique_entity']] = group_similar_strings(dataframe['clean_names'], min_similarity = 0.2)
    dataframe = dataframe.groupby(['unique_entity_ID', 'unique_entity'])[column_name].apply('; '.join).reset_index()
    return dataframe

In [47]:
df_unique_entities_names = entity_matcher_names(df_names_cleaned, 'names'); df_unique_entities_names

,unique_entity_ID,unique_entity,names
0,0,marks and spencers,Marks and Spencers Ltd
1,1,ms,M&S Limited
2,2,nvidia ireland,NVIDIA Ireland
3,3,intel,Intel LLC
4,4,at t,AT & T; AT & T MOBILITY; AT&T; AT&T MOBILITY
5,8,dell,DELL INC.
6,9,walmart community,WAL-MART COMMUNITY; WALMART PAYMENTS
7,11,pitney bowes,PITNEY BOWES; PITNEY BOWES GLOBAL FINANCIAL; P...
8,14,oracle,ORACLE; ORACLE AMERICA INC; ORACLE CORPORATION...


In [48]:
# function for processing df of names and clustering them using string similarity
def clustering_names(dataframe, column_name):
    # preprocess dataframe
    df_names_cleaned = preprocess_names(dataframe, column_name)
    # start clustering of names
    print('Entity clustering will start now.')
    start = time.time()
    df_unique_entities_names = entity_matcher_names(df_names_cleaned, column_name)
    time.sleep(1)
    end = time.time()
    # print total time taken
    print(f"Runtime of the program is {end - start} seconds.")
    # create csv with timestamp
    current_time = time.strftime("%Y%m%d-%H%M%S")
    filename = 'names_normalized' + str(current_time) + '.csv'
    # print job status
    print(f"{filename} has been saved in current directory.")
    return df_unique_entities_names.to_csv(filename, index = False)

In [49]:
clustering_names(df_names, 'names')

Entity clustering will start now.
Runtime of the program is 1.016322135925293 seconds.
names_normalized20210725-235626.csv has been saved in current directory.


In [50]:
# inspect results
pd.read_csv('names_normalized20210725-235626.csv')

,unique_entity_ID,unique_entity,names
0,0,marks and spencers,Marks and Spencers Ltd
1,1,ms,M&S Limited
2,2,nvidia ireland,NVIDIA Ireland
3,3,intel,Intel LLC
4,4,at t,AT & T; AT & T MOBILITY; AT&T; AT&T MOBILITY
5,8,dell,DELL INC.
6,9,walmart community,WAL-MART COMMUNITY; WALMART PAYMENTS
7,11,pitney bowes,PITNEY BOWES; PITNEY BOWES GLOBAL FINANCIAL; P...
8,14,oracle,ORACLE; ORACLE AMERICA INC; ORACLE CORPORATION...


We got issues with M&S Limited unfortunately. In this particular case, the google search scraped link would have performed better than the approach we decided to go with.

## Part 2: General Normalization Engine

My first thought in this part was to use the approach outlined in part 1, with the addition of Named Entity Recognition (NER) right after the text pre-processing step. This would allow to separate all strings into their respective categories (serial numbers, physical goods, locations, company addresses, company names) and would allow us to use the category-specific normalization engines that were built in part 1. The issues are that NER works best when words have a context within a sentence and entity types such as serial numbers and addresses would have to be custom-trained. We would also still be relying on the rule-based cleaning engines developed in part 1 instead of making use of intelligent systems.

In thinking about a solution, I think it could be interesting to explore NER with the spaCy library. An [open-source annotation tool](https://github.com/ManivannanMurugavel/spacy-ner-annotator) can be used to create a dataset that we could use to fine-tune a spaCy NER model. When I say fine-tuning, there are 2 possibilities that both seem like risky hacks: 

Option 1 - Creating new, custom entity types for the entities that don't yet have an entity type (serial number, address, physical goods) and updating a model with these new types. The obvious risk is that we will be run into issues of conflicting entity types. For example ADDRESS vs LOCATION.

Option 2 - This one sounds quite ridiculous. We do not create custom entity types and instead train the model to recognize serial numbers as persons (PER) for example, making sure each category (serial numbers, physical goods, locations, company addresses, company names) have a unique entity type to categorize them.

We could also decide to not fine tune and instead start with a blank model and teach it new entity types. Beyond this NER task, we need to start thinking about alternatives to the rule-based cleaning engines. 

Other ideas to explore:

- Try brute-force approach of grouping all entities, completely ignoring semantics. This will give us a baseline of the performance to beat
- Explore use of Flair library for NER, confidence score is a feature that could be useful.
- [Wikipedia paragraph classification](https://github.com/yashsmehta/named-entity-normalization): Scrape a paragraph from wikipedia describing the entity in question, pass the paragraph through a language model (ie: BERT), feed embedding vector to a shallow classifier to make the prediction of what entity class the new entity belongs to. Could use fastai library for super quick setup/execution.
- Re-explore clustering-based methods for entity normalization.
- Explore viability of Named Entity Linking using Facebook's BLINK library or spaCy.
- Create large dataset for all categories, assemble from all datasets found online so far.
- Dealing with incoming strings: seems like the current approach will hold up with incoming strings, shouldn't worry too much about this.